# Install

In [ ]:
!pip3 install swifter

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install pip==20.0.2 pororo fairseq==0.10.2

In [ ]:
!pip uninstall torch torchvision

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu   # cpuonly

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.13.1+cpu
cuda version: None
cudnn version:None


# Data Load

In [10]:
import os
os.chdir('/content/drive/MyDrive/Mbti_LexicalTendency/src')

import config as cfg

In [11]:
from pprint import pprint
import pandas as pd
from pshmodule.utils import filemanager as fm

In [12]:
nf = pd.read_excel(cfg.nf, engine="openpyxl")
# st = pd.read_excel(cfg.st, engine="openpyxl", header=1)

In [13]:
nf.head()

,speaker,uttterance
0,화자1,뭘로 해볼까요?
1,화자2,음 논리적 말하기
2,화자2,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데
3,화자1,그냥 자신의 생각을 설명해도 될 거 같은데요.
4,화자1,뭔가 일 얘기는 하고 싶지 않아.


In [15]:
nf.shape

(1277, 2)

In [16]:
nf.dropna(inplace=True)

In [18]:
nf.shape

(1276, 2)

In [19]:
nf = nf[['speaker', 'uttterance']]
# st = st[['speaker', 'uttterance']]

In [21]:
nf.rename(columns={'uttterance':'utterance'}, inplace=True)
# st.rename(columns={'uttterance':'utterance'}, inplace=True)

In [22]:
nf.head()

,speaker,utterance
0,화자1,뭘로 해볼까요?
1,화자2,음 논리적 말하기
2,화자2,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데
3,화자1,그냥 자신의 생각을 설명해도 될 거 같은데요.
4,화자1,뭔가 일 얘기는 하고 싶지 않아.


# Pororo

In [23]:
import swifter
from pororo import Pororo

In [24]:
Pororo.available_models("dp")
dp = Pororo(task="dep_parse", lang="ko")

In [25]:
nf['pororo'] = nf.utterance.swifter.apply(lambda x: dp(x))
# st['pororo'] = st.utterance.swifter.apply(lambda x: dp(x))

Pandas Apply:   0%|          | 0/1276 [00:00<?, ?it/s]

# NF

다른 mbti 나오면 배열 처리 ㄱㄱ

In [26]:
# window size
data_list = [2, 3, 4, 5, 6]

for num in data_list:
  exec(f'result_{str(num)} = []')
  for i in nf.iterrows():
    for j in range(num-1, len(i[1]['pororo'])):
      temp = []
      temp.append(i[1]['utterance'])
      if num == 6:
        # token6
        temp.append(i[1]['pororo'][j-5][1])
        temp.append(i[1]['pororo'][j-5][3])
        temp.append(i[1]['pororo'][j-4][1])
        temp.append(i[1]['pororo'][j-4][3])
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      elif num == 5:
        # token5
        temp.append(i[1]['pororo'][j-4][1])
        temp.append(i[1]['pororo'][j-4][3])        
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])
        
      elif num == 4:
        # token4
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      elif num == 3:
        # token3
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      # token2
      temp.append(i[1]['pororo'][j-1][1])
      temp.append(i[1]['pororo'][j-1][3])      
      # token1
      temp.append(i[1]['pororo'][j][1])
      temp.append(i[1]['pororo'][j][3])
      
      exec(f'result_{str(num)}.append({temp})')

In [27]:
print(f"result_2 : {result_2[:5]}")
print(f"result_3 : {result_3[:5]}")
print(f"result_4 : {result_4[:5]}")
print(f"result_5 : {result_5[:5]}")
print(f"result_6 : {result_6[:5]}")

result_2 : [['뭘로 해볼까요?', '뭘로', 'NP_OBJ', '해볼까요?', 'VP'], ['음 논리적 말하기', '음', 'IP', '논리적', 'NP_OBJ'], ['음 논리적 말하기', '논리적', 'NP_OBJ', '말하기', 'VP'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '논리적', 'NP', '말하기에서', 'VP_AJT'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '말하기에서', 'VP_AJT', '아', 'VP']]
result_3 : [['음 논리적 말하기', '음', 'IP', '논리적', 'NP_OBJ', '말하기', 'VP'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '논리적', 'NP', '말하기에서', 'VP_AJT', '아', 'VP'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '말하기에서', 'VP_AJT', '아', 'VP', '근데', 'AP'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '아', 'VP', '근데', 'AP', '나는', 'NP_SBJ'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '근데', 'AP', '나는', 'NP_SBJ', '생', 'NP_AJT']]
result_4 : [['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '논리적', 'NP', '말하기에서', 'VP_AJT', '아', 'VP', '근데', 'AP'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '말하기에서', 'VP_AJT', '아', 'VP', '근데', 'AP', '나는', 'NP_SBJ'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '아', 'VP', '근데', 'AP', '나는', 'NP_SBJ', '생', 'NP_AJT'], ['논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데', '근데', 'AP', '나는', 'N

In [28]:
frames = {
  'nf2': "pd.DataFrame(result_2, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag']])",
  'nf3': "pd.DataFrame(result_3, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag']])",
  'nf4': "pd.DataFrame(result_4, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag']])",
  'nf5': "pd.DataFrame(result_5, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag', 'token5', 'token5_tag']])",
  'nf6': "pd.DataFrame(result_6, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag', 'token5', 'token5_tag', 'token6', 'token6_tag']])",
}

for name, df in frames.items():
  exec(f'{name} = {df}')

In [33]:
nf6.head()

,utterance,token1,token1_tag,token2,token2_tag,token3,token3_tag,token4,token4_tag,token5,token5_tag,token6,token6_tag
0,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데,논리적,NP,말하기에서,VP_AJT,아,VP,근데,AP,나는,NP_SBJ,생,NP_AJT
1,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데,말하기에서,VP_AJT,아,VP,근데,AP,나는,NP_SBJ,생,NP_AJT,설득하는,VP_MOD
2,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데,아,VP,근데,AP,나는,NP_SBJ,생,NP_AJT,설득하는,VP_MOD,거,NP_SBJ
3,논리적 말하기에서 아 근데 나는 생 설득하는 거 힘든데,근데,AP,나는,NP_SBJ,생,NP_AJT,설득하는,VP_MOD,거,NP_SBJ,힘든데,VP
4,그냥 자신의 생각을 설명해도 될 거 같은데요.,그냥,AP,자신의,NP_MOD,생각을,NP_OBJ,설명해도,VP,될,VP_MOD,거,NP


# Save

In [37]:
import os
os.chdir('/content/drive/MyDrive/Mbti_LexicalTendency/src')

In [38]:
import config as cfg

In [39]:
with pd.ExcelWriter(cfg.nf_save) as writer:
    nf2.to_excel(writer, sheet_name = 'ngram2')
    nf3.to_excel(writer, sheet_name = 'ngram3')
    nf4.to_excel(writer, sheet_name = 'ngram4')
    nf5.to_excel(writer, sheet_name = 'ngram5')
    nf6.to_excel(writer, sheet_name = 'ngram6')